In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from dataset import defectDataset_convolution
import random
import math
import seaborn
from  matplotlib import pyplot
import torch.nn.functional as F
import matplotlib.image as mpimg
import torch.nn as nn
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
import util
from util import *

In [13]:
window_size = 0.15
valid = np.concatenate((np.arange(6501,7001),np.arange(8201,8234),np.arange(8801,8834)))
test = np.concatenate((np.arange(7001,8001),np.arange(8234,8301),np.arange(8834,8901)))
test = np.arange(6501,8001)

In [14]:
df_yolo = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_yolo.csv', sep=' ')
df_truth = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/test_modified.csv',index_col=0)
# df_truth = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/test.csv', sep=' ')
df_MMN = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_MMN.csv', sep=' ')
columns = ['image_index','class', 'x', 'y']
df_yolo_points = pd.DataFrame(index=np.arange(len(df_yolo)), columns=columns)
df_yolo_points['image_index'] = df_yolo['image_index']
df_yolo_points['class'] = df_yolo['class']
df_yolo_points['x'] = (df_yolo['x1'] + df_yolo['x2'])/2
df_yolo_points['y'] = (df_yolo['y1'] + df_yolo['y2'])/2
for index, row in df_yolo.iterrows():
    if (row['class'] == 0) & (row.confidence<0.24):
        df_yolo_points = df_yolo_points.drop(index)
    if (row['class'] == 1) & (row.confidence<0.1):
        df_yolo_points = df_yolo_points.drop(index)
    if (row['class'] == 2) & (row.confidence<0.12):
        df_yolo_points = df_yolo_points.drop(index)
    if (row['class'] == 3) & (row.confidence<0.06):
        df_yolo_points = df_yolo_points.drop(index)
df_MMN = df_MMN.drop(columns = ['confidence'])
new_order = [1,0,2,3]
df_MMN = df_MMN[df_MMN.columns[new_order]]
df_yolo_points = df_yolo_points.replace({'class':2},0)
# df_yolo_points.to_csv('/home/rliu/github/defect_classifier/results/yolo_1-19-18.csv', sep=' ', index=False, header=False)

In [15]:
columns = ['image_index','class', 'x', 'y']
df_results = pd.DataFrame(index=np.arange(0), columns=columns)
# df_truth = df_truth[(df_truth['image_index'] <= 8900) & (df_truth['image_index'] >= 8801)]
# df_test = df_truth.loc[df_truth['image_index'].isin(np.arange(6501,6601))]
for i in test:# detecting local maxima in confidence map to find defects
    columns = ['image_index','class', 'x', 'y']
    df_img = pd.DataFrame(index=np.arange(0), columns=columns)
    
    arr_pos = np.load('/home/rliu/defect_classifier/convolution_test/pos/%06.0f.npy' % i)
    arr_neg = np.load('/home/rliu/defect_classifier/convolution_test/neg/%06.0f.npy' % i)
    arr_pos_o = np.load('/home/rliu/defect_classifier/convolution_test/pos_o/%06.0f.npy' % i)
    arr_nuc = np.load('/home/rliu/defect_classifier/convolution_test/nuc/%06.0f.npy' % i)
    
    coord_pos = peak_local_max(arr_pos, min_distance=12, threshold_abs=0.98, threshold_rel = 0.9).astype(float)
    coord_neg = peak_local_max(arr_neg, min_distance=12, threshold_abs=0.975, threshold_rel = 0.9).astype(float)
    coord_pos_o = peak_local_max(arr_pos_o, min_distance=12, threshold_abs=0.97, threshold_rel = 0.9).astype(float)
    coord_nuc = peak_local_max(arr_nuc, min_distance=12, threshold_abs=0.93, threshold_rel = 0.9).astype(float)
    
    coord_pos[:,0], coord_pos[:,1] = coord_pos[:,0]/150.0, coord_pos[:,1]/150.0
    coord_neg[:,0], coord_neg[:,1] = coord_neg[:,0]/150.0, coord_neg[:,1]/150.0
    coord_pos_o[:,0], coord_pos_o[:,1] = coord_pos_o[:,0]/150.0, coord_pos_o[:,1]/150.0
    coord_nuc[:,0], coord_nuc[:,1] = coord_nuc[:,0]/150.0, coord_nuc[:,1]/150.0
    
    df_pos = pd.DataFrame(index=np.arange(coord_pos.shape[0]), columns=columns)
    df_neg = pd.DataFrame(index=np.arange(coord_neg.shape[0]), columns=columns)
    df_pos_o = pd.DataFrame(index=np.arange(coord_pos_o.shape[0]), columns=columns)
    df_nuc = pd.DataFrame(index=np.arange(coord_nuc.shape[0]), columns=columns)
    
    df_pos['class'] = 0
    df_neg['class'] = 1
    df_pos_o['class'] = 2
    df_nuc['class'] = 3
    
    df_pos['x'], df_pos['y'] = coord_pos[:,1], 1 - coord_pos[:,0]
    df_neg['x'], df_neg['y'] = coord_neg[:,1], 1 - coord_neg[:,0]
    df_pos_o['x'], df_pos_o['y'] = coord_pos_o[:,1], 1 - coord_pos_o[:,0]
    df_nuc['x'], df_nuc['y'] = coord_nuc[:,1], 1 - coord_nuc[:,0]
    
    df_img = df_img.append(df_pos, ignore_index=True)
    df_img = df_img.append(df_neg, ignore_index=True)
    df_img = df_img.append(df_pos_o, ignore_index=True)
    df_img = df_img.append(df_nuc, ignore_index=True)
    
    df_img['image_index'] = i
    df_results = df_results.append(df_img, ignore_index=True)

In [16]:
df_results_yolo = checkYolo(df_yolo = df_yolo.replace({'class':2},0), df_results = df_results.replace({'class':2},0), conf_thres_pos = 0.001, conf_thres_neg = 0.001, conf_thres_nuc = 0.001, conf_thres_pos_o = 0.001)
df_results_yolo = putBackYOLO(window_size = 0.20, conf_thres_pos = 0.7, conf_thres_neg = 0.8, conf_thres_nuc = 1, conf_thres_pos_o = 0.9, df_yolo = df_yolo, df_results = df_results_yolo.replace({'class':2},0))
df_truth = df_truth.loc[df_truth['image_index'].isin(test)]
df_results_yolo = df_results_yolo.replace({'class':2},0)
df = compareLabels(df_results_yolo, df_truth.replace({'class':2},0), window_size=window_size)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

   truth_tot  truth_pos  truth_neg  truth_pos_o  truth_nuc  correct_tot  \
0       8018       5039       2274            0        705         5747   

   correct_pos  correct_neg  correct_pos_o  correct_nuc  pred_tot  pred_pos  \
0         4049         1399              0          299      7434      4682   

   pred_neg  pred_pos_o  pred_nuc     dis_pos     dis_neg  dis_pos_o  \
0      2090           0       662  311.470693  522.880141          0   

      dis_nuc  dis_pos_c  dis_neg_c  dis_pos_o_c  dis_nuc_c  
0  309.127822  67.248952  13.908682            0   3.388086  


In [17]:
precision_tot = df.loc[0,'correct_tot'] / df.loc[0,'pred_tot']
precision_pos = df.loc[0,'correct_pos'] / df.loc[0,'pred_pos']
precision_neg = df.loc[0,'correct_neg'] / df.loc[0,'pred_neg']
precision_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'pred_pos_o']
precision_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'pred_nuc']

recall_tot = df.loc[0,'correct_tot'] / df.loc[0,'truth_tot']
recall_pos = df.loc[0,'correct_pos'] / df.loc[0,'truth_pos']
recall_neg = df.loc[0,'correct_neg'] / df.loc[0,'truth_neg']
recall_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'truth_pos_o']
recall_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'truth_nuc']

accuracy_tot = 2*(precision_tot*recall_tot)/(precision_tot+recall_tot)
accuracy_pos = 2*(precision_pos*recall_pos)/(precision_pos+recall_pos)
accuracy_neg = 2*(precision_neg*recall_neg)/(precision_neg+recall_neg)
accuracy_pos_o = 2*(precision_pos_o*recall_pos_o)/(precision_pos_o+recall_pos_o)
accuracy_nuc = 2*(precision_nuc*recall_nuc)/(precision_nuc+recall_nuc)

precision_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'pred_pos']+df.loc[0,'pred_neg'])
recall_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'truth_pos']+df.loc[0,'truth_neg'])
accuracy_pn = 2*(precision_pn*recall_pn)/(precision_pn+recall_pn)

dis_pos_c = df.loc[0,'dis_pos_c'] / df.loc[0,'correct_pos']
dis_neg_c = df.loc[0,'dis_neg_c'] / df.loc[0,'correct_neg']
dis_pos_o_c = df.loc[0,'dis_pos_o_c'] / df.loc[0,'correct_pos_o']
dis_nuc_c = df.loc[0,'dis_nuc_c'] / df.loc[0,'correct_nuc']

print('precision_tot =', precision_tot)
print('precision_pos =', precision_pos)
print('precision_neg =', precision_neg)
print('precision_pos_o =', precision_pos_o)
print('precision_nuc =', precision_nuc)
print('precision_pn =', precision_pn)
print('\n')
print('recall_tot =', recall_tot)
print('recall_pos =', recall_pos)
print('recall_neg =', recall_neg)
print('recall_pos_o =', recall_pos_o)
print('recall_nuc =', recall_nuc)
print('recall_pn =', recall_pn)
print('\n')
print('accuracy_tot =', accuracy_tot)
print('accuracy_pos =', accuracy_pos)
print('accuracy_neg =', accuracy_neg)
print('accuracy_pos_o =', accuracy_pos_o)
print('accuracy_nuc =', accuracy_nuc)
print('\n')
print('dis_pos =', dis_pos_c)
print('dis_neg =', dis_neg_c)
print('dis_pos_o =', dis_pos_o_c)
print('dis_nuc =', dis_nuc_c)
print('\n')
print('accuracy_pn =', accuracy_pn)

precision_tot = 0.7730696798493408
precision_pos = 0.8648013669372063
precision_neg = 0.669377990430622
precision_pos_o = nan
precision_nuc = 0.4516616314199396
precision_pn = 0.8044890726520969


recall_tot = 0.7167622848590671
recall_pos = 0.8035324469140702
recall_neg = 0.6152154793315743
recall_pos_o = nan
recall_nuc = 0.42411347517730497
recall_pn = 0.7449747025844387


accuracy_tot = 0.7438519285529381
accuracy_pos = 0.8330418681205637
accuracy_neg = 0.6411549037580201
accuracy_pos_o = nan
accuracy_nuc = 0.43745427944403803


dis_pos = 0.01660878045487083
dis_neg = 0.009941873983481684
dis_pos_o = nan
dis_nuc = 0.011331390192139525


accuracy_pn = 0.7735889243876465


/home/rliu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.
/home/rliu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.
/home/rliu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars


In [85]:
df_results_post = connect_detection(df_results = df_results_yolo, inspect_range = 16, inspect_radius = 0.05, inspect_thres_low = 0.3, inspect_thres_high = 0.65)
df = util.compareLabels(df_results_post, df_truth.replace({'class':2},0), window_size=window_size)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

   truth_tot  truth_pos  truth_neg  truth_pos_o  truth_nuc  correct_tot  \
0       8018       5039       2274            0        705         5981   

   correct_pos  correct_neg  correct_pos_o  correct_nuc  pred_tot  pred_pos  \
0         4178         1504              0          299      7920      4952   

   pred_neg  pred_pos_o  pred_nuc     dis_pos    dis_neg  dis_pos_o  \
0      2306           0       662  266.319657  475.68168          0   

      dis_nuc  dis_pos_c  dis_neg_c  dis_pos_o_c  dis_nuc_c  
0  309.127822  63.709587  15.964523            0   3.388086  


In [86]:
precision_tot = df.loc[0,'correct_tot'] / df.loc[0,'pred_tot']
precision_pos = df.loc[0,'correct_pos'] / df.loc[0,'pred_pos']
precision_neg = df.loc[0,'correct_neg'] / df.loc[0,'pred_neg']
precision_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'pred_pos_o']
precision_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'pred_nuc']

recall_tot = df.loc[0,'correct_tot'] / df.loc[0,'truth_tot']
recall_pos = df.loc[0,'correct_pos'] / df.loc[0,'truth_pos']
recall_neg = df.loc[0,'correct_neg'] / df.loc[0,'truth_neg']
recall_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'truth_pos_o']
recall_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'truth_nuc']

accuracy_tot = 2*(precision_tot*recall_tot)/(precision_tot+recall_tot)
accuracy_pos = 2*(precision_pos*recall_pos)/(precision_pos+recall_pos)
accuracy_neg = 2*(precision_neg*recall_neg)/(precision_neg+recall_neg)
accuracy_pos_o = 2*(precision_pos_o*recall_pos_o)/(precision_pos_o+recall_pos_o)
accuracy_nuc = 2*(precision_nuc*recall_nuc)/(precision_nuc+recall_nuc)

precision_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'pred_pos']+df.loc[0,'pred_neg'])
recall_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'truth_pos']+df.loc[0,'truth_neg'])
accuracy_pn = 2*(precision_pn*recall_pn)/(precision_pn+recall_pn)

dis_pos_c = df.loc[0,'dis_pos_c'] / df.loc[0,'correct_pos']
dis_neg_c = df.loc[0,'dis_neg_c'] / df.loc[0,'correct_neg']
dis_pos_o_c = df.loc[0,'dis_pos_o_c'] / df.loc[0,'correct_pos_o']
dis_nuc_c = df.loc[0,'dis_nuc_c'] / df.loc[0,'correct_nuc']



print('precision_tot =', precision_tot)
print('precision_pos =', precision_pos)
print('precision_neg =', precision_neg)
print('precision_pos_o =', precision_pos_o)
print('precision_nuc =', precision_nuc)
print('precision_pn =', precision_pn)
print('\n')
print('recall_tot =', recall_tot)
print('recall_pos =', recall_pos)
print('recall_neg =', recall_neg)
print('recall_pos_o =', recall_pos_o)
print('recall_nuc =', recall_nuc)
print('recall_pn =', recall_pn)
print('\n')
print('accuracy_tot =', accuracy_tot)
print('accuracy_pos =', accuracy_pos)
print('accuracy_neg =', accuracy_neg)
print('accuracy_pos_o =', accuracy_pos_o)
print('accuracy_nuc =', accuracy_nuc)
print('\n')
print('dis_pos =', dis_pos_c)
print('dis_neg =', dis_neg_c)
print('dis_pos_o =', dis_pos_o_c)
print('dis_nuc =', dis_nuc_c)
print('\n')
print('accuracy_pn =', accuracy_pn)

precision_tot = 0.7551767676767677
precision_pos = 0.8436995153473344
precision_neg = 0.6522116218560278
precision_pos_o = nan
precision_nuc = 0.4516616314199396
precision_pn = 0.7828602920914852


recall_tot = 0.7459466201047643
recall_pos = 0.8291327644373884
recall_neg = 0.6613896218117854
recall_pos_o = nan
recall_nuc = 0.42411347517730497
recall_pn = 0.7769725146998496


accuracy_tot = 0.7505333166018321
accuracy_pos = 0.8363527174457012
accuracy_neg = 0.656768558951965
accuracy_pos_o = nan
accuracy_nuc = 0.43745427944403803


dis_pos = 0.015248824001200957
dis_neg = 0.010614709491770107
dis_pos_o = nan
dis_nuc = 0.011331390192139525


accuracy_pn = 0.779905291332098


/home/rliu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.
/home/rliu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.
/home/rliu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars


In [30]:
df_results_post

,image_index,class,x,y
0,6501.0,0.0,0.733333,0.213333
1,6501.0,0.0,0.453333,0.393333
2,6501.0,0.0,0.246667,0.560000
3,6501.0,0.0,0.480000,0.766667
4,6501.0,1.0,0.160000,0.606667
5,6502.0,0.0,0.733333,0.220000
6,6502.0,0.0,0.446667,0.393333
7,6502.0,0.0,0.246667,0.560000
8,6502.0,0.0,0.486667,0.766667
9,6502.0,1.0,0.086667,0.506667
